In [2]:
from datetime import datetime
import pandas as pd
import requests
import json
import boto3
import sqlalchemy

from requests.auth import HTTPBasicAuth


In [3]:
url = "https://diaad.s3.sa-east-1.amazonaws.com/sim/Mortalidade_Geral_2020.csv"

In [4]:
df2 = pd.read_csv(url,sep=';')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (66,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df = df2
df.head()

,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR
0,1,2,18052020,1452.0,812.0,120010.0,7061932.0,487,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603
1,1,2,20052020,2115.0,812.0,120010.0,16031952.0,468,2,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,604
2,1,2,21052020,1200.0,812.0,120010.0,17021961.0,459,2,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605
3,1,2,21052020,1233.0,812.0,120010.0,10081942.0,477,2,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,606
4,1,2,22052020,730.0,812.0,120030.0,13041936.0,484,2,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607


In [7]:
df.columns = df.columns.str.lower()
df.columns

Index(['origem', 'tipobito', 'dtobito', 'horaobito', 'natural', 'codmunnatu',
       'dtnasc', 'idade', 'sexo', 'racacor', 'estciv', 'esc', 'esc2010',
       'seriescfal', 'ocup', 'codmunres', 'lococor', 'codestab', 'estabdescr',
       'codmunocor', 'idademae', 'escmae', 'escmae2010', 'seriescmae',
       'ocupmae', 'qtdfilvivo', 'qtdfilmort', 'gravidez', 'semagestac',
       'gestacao', 'parto', 'obitoparto', 'peso', 'tpmorteoco', 'obitograv',
       'obitopuerp', 'assistmed', 'exame', 'cirurgia', 'necropsia', 'linhaa',
       'linhab', 'linhac', 'linhad', 'linhaii', 'causabas', 'cb_pre',
       'comunsvoim', 'dtatestado', 'circobito', 'acidtrab', 'fonte',
       'numerolote', 'tppos', 'dtinvestig', 'causabas_o', 'dtcadastro',
       'atestante', 'stcodifica', 'codificado', 'versaosist', 'versaoscb',
       'fonteinv', 'dtrecebim', 'atestado', 'dtrecoriga', 'causamat',
       'escmaeagr1', 'escfalagr1', 'stdoepidem', 'stdonova', 'difdata',
       'nudiasobco', 'nudiasobin', 'dtcadinv

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556824 entries, 0 to 1556823
Data columns (total 87 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   origem      1556824 non-null  int64  
 1   tipobito    1556824 non-null  int64  
 2   dtobito     1556824 non-null  int64  
 3   horaobito   1497806 non-null  float64
 4   natural     1469096 non-null  float64
 5   codmunnatu  1446983 non-null  float64
 6   dtnasc      1553080 non-null  float64
 7   idade       1556824 non-null  int64  
 8   sexo        1556824 non-null  int64  
 9   racacor     1516188 non-null  float64
 10  estciv      1475822 non-null  float64
 11  esc         1442466 non-null  float64
 12  esc2010     1437863 non-null  float64
 13  seriescfal  361095 non-null   float64
 14  ocup        1306818 non-null  float64
 15  codmunres   1556824 non-null  int64  
 16  lococor     1556824 non-null  int64  
 17  codestab    1131930 non-null  float64
 18  estabdescr  0 non-null

In [9]:
dateColumns = ['dtobito','dtnasc','dtatestado','dtinvestig','dtrecoriga'
,'dtcadinv','dtconinv','dtcadinf','dtconcaso']

In [10]:
df[dateColumns[:]].head(3)

,dtobito,dtnasc,dtatestado,dtinvestig,dtrecoriga,dtcadinv,dtconinv,dtcadinf,dtconcaso
0,18052020,7061932.0,18052020.0,NaN,29052020,NaN,NaN,NaN,NaN
1,20052020,16031952.0,20052020.0,4062020.0,29052020,NaN,NaN,NaN,NaN
2,21052020,17021961.0,21052020.0,NaN,29052020,NaN,NaN,NaN,NaN


In [11]:
for icol in dateColumns :
    df[icol] = pd.to_datetime(df[icol], format='%d%m%Y',errors='coerce')

In [12]:
df[dateColumns[:]].head(3)

,dtobito,dtnasc,dtatestado,dtinvestig,dtrecoriga,dtcadinv,dtconinv,dtcadinf,dtconcaso
0,2020-05-18,1932-06-07,2020-05-18,NaT,2020-05-29,NaT,NaT,NaT,NaT
1,2020-05-20,1952-03-16,2020-05-20,2020-06-04,2020-05-29,NaT,NaT,NaT,NaT
2,2020-05-21,1961-02-17,2020-05-21,NaT,2020-05-29,NaT,NaT,NaT,NaT


Agora queremos arrumar o campo de hora. Observe que a coluna 'horaobito' apresenta os campos de hora e minuto concatenados e estão exibidos em formato float64. Queremos transformá-lo para concaternas com a data de dtobito e termos um campos datetime de fato. 

In [18]:
df["horaobito"] = pd.to_datetime(df['horaobito'], format='%H%M',errors='coerce')

In [19]:
df.head(4)

,origem,tipobito,dtobito,horaobito,natural,codmunnatu,dtnasc,idade,sexo,racacor,...,fontes,tpresginfo,tpnivelinv,nudiasinf,dtcadinf,morteparto,dtconcaso,fontesinf,altcausa,contador
0,1,2,2020-05-18,1900-01-01 14:52:00,812.0,120010.0,1932-06-07,487,1,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,603
1,1,2,2020-05-20,1900-01-01 21:15:00,812.0,120010.0,1952-03-16,468,2,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,604
2,1,2,2020-05-21,1900-01-01 12:00:00,812.0,120010.0,1961-02-17,459,2,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,605
3,1,2,2020-05-21,1900-01-01 12:33:00,812.0,120010.0,1942-08-10,477,2,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,606


In [20]:
df['dtobito'] = pd.to_datetime(df['dtobito'].astype(str)+' '+df['horaobito'].dt.time.astype(str),errors='coerce')

In [21]:
df.head()

,origem,tipobito,dtobito,horaobito,natural,codmunnatu,dtnasc,idade,sexo,racacor,...,fontes,tpresginfo,tpnivelinv,nudiasinf,dtcadinf,morteparto,dtconcaso,fontesinf,altcausa,contador
0,1,2,2020-05-18 14:52:00,1900-01-01 14:52:00,812.0,120010.0,1932-06-07,487,1,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,603
1,1,2,2020-05-20 21:15:00,1900-01-01 21:15:00,812.0,120010.0,1952-03-16,468,2,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,604
2,1,2,2020-05-21 12:00:00,1900-01-01 12:00:00,812.0,120010.0,1961-02-17,459,2,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,605
3,1,2,2020-05-21 12:33:00,1900-01-01 12:33:00,812.0,120010.0,1942-08-10,477,2,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,606
4,1,2,2020-05-22 07:30:00,1900-01-01 07:30:00,812.0,120030.0,1936-04-13,484,2,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,607


In [22]:
df = df.drop(['horaobito'],axis=1)

In [23]:
df.head(4)

,origem,tipobito,dtobito,natural,codmunnatu,dtnasc,idade,sexo,racacor,estciv,...,fontes,tpresginfo,tpnivelinv,nudiasinf,dtcadinf,morteparto,dtconcaso,fontesinf,altcausa,contador
0,1,2,2020-05-18 14:52:00,812.0,120010.0,1932-06-07,487,1,4.0,3.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,603
1,1,2,2020-05-20 21:15:00,812.0,120010.0,1952-03-16,468,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,604
2,1,2,2020-05-21 12:00:00,812.0,120010.0,1961-02-17,459,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,605
3,1,2,2020-05-21 12:33:00,812.0,120010.0,1942-08-10,477,2,4.0,9.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,606


In [41]:
df['idade'] = (((df['dtobito'] - df['dtnasc']).dt.days)/365)
#print(type(df['dtnasc'][0]),type(df['dtobito'][0]))

In [45]:
df.head(3)

,origem,tipobito,dtobito,natural,codmunnatu,dtnasc,idade,sexo,racacor,estciv,...,fontes,tpresginfo,tpnivelinv,nudiasinf,dtcadinf,morteparto,dtconcaso,fontesinf,altcausa,contador
0,1,2,2020-05-18 14:52:00,812.0,120010.0,1932-06-07,88.005479,1,4.0,3.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,603
1,1,2,2020-05-20 21:15:00,812.0,120010.0,1952-03-16,68.224658,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,604
2,1,2,2020-05-21 12:00:00,812.0,120010.0,1961-02-17,59.295890,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,605


In [47]:
import numpy as np
df['idade'] = np.floor(pd.to_numeric(df['idade'], errors='coerce')).astype('Int64') 
#df['idade'].astype('Int64')

In [48]:
df

,origem,tipobito,dtobito,natural,codmunnatu,dtnasc,idade,sexo,racacor,estciv,...,fontes,tpresginfo,tpnivelinv,nudiasinf,dtcadinf,morteparto,dtconcaso,fontesinf,altcausa,contador
0,1,2,2020-05-18 14:52:00,812.0,120010.0,1932-06-07,88,1,4.0,3.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,603
1,1,2,2020-05-20 21:15:00,812.0,120010.0,1952-03-16,68,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,604
2,1,2,2020-05-21 12:00:00,812.0,120010.0,1961-02-17,59,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,605
3,1,2,2020-05-21 12:33:00,812.0,120010.0,1942-08-10,77,2,4.0,9.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,606
4,1,2,2020-05-22 07:30:00,812.0,120030.0,1936-04-13,84,2,4.0,3.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556819,1,2,2020-12-31 20:20:00,831.0,316960.0,1929-07-18,91,2,4.0,4.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,1553493
1556820,1,2,2020-12-31 12:20:00,817.0,172100.0,2020-12-07,0,1,1.0,NaN,...,SXXSXX,NaN,NaN,NaN,2021-02-12,3.0,2021-02-09,NaN,2.0,1553522
1556821,1,2,NaT,826.0,261580.0,1955-10-30,<NA>,1,2.0,9.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,1553533
1556822,1,2,2020-12-31 21:00:00,841.0,410830.0,1982-12-16,38,1,4.0,5.0,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,1553534


Neste ponto ja temos o dataframe praticamente como queremos. O que nos falta agora é converter campos que são mostrados como float mas que são, na realidade, inteiros. Por exemplo 'codmunnatu'

In [19]:
date_fields =  ['dtnasc','dtatestado','dtinvestig','dtrecoriga','dtcadinv','dtconinv','dtcadinf','dtconcaso']

df['dtrecoriga'].where(df['dtregoriga'].notna() == True)

0          18052020.0
1          20052020.0
2          21052020.0
3          21052020.0
4          22052020.0
              ...    
1556819    31122020.0
1556820    31122020.0
1556821     2012021.0
1556822     1012021.0
1556823    31122020.0
Name: dtatestado, Length: 1556824, dtype: float64

In [13]:
df = df.drop(['horaobito'],axis=1)
df.head()

,origem,tipobito,dtobito,natural,codmunnatu,dtnasc,idade,sexo,racacor,estciv,...,fontes,tpresginfo,tpnivelinv,nudiasinf,dtcadinf,morteparto,dtconcaso,fontesinf,altcausa,contador
0,1,2,2020-05-18 14:52:00,812.0,120010.0,1932-06-07,487,1,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603
1,1,2,2020-05-20 21:15:00,812.0,120010.0,1952-03-16,468,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,604
2,1,2,2020-05-21 12:00:00,812.0,120010.0,1961-02-17,459,2,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605
3,1,2,2020-05-21 12:33:00,812.0,120010.0,1942-08-10,477,2,4.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,606
4,1,2,2020-05-22 07:30:00,812.0,120030.0,1936-04-13,484,2,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607


In [14]:
df['natural']    = df['natural'].astype('Int64')
df['codmunnatu'] = df['codmunnatu'].astype('Int64')
df['racacor']    = df['racacor'].astype('Int64')
df['estciv']     = df['estciv'].astype('Int64')
df['ocup']       = df['ocup'].astype('Int64')
df['codmunres']  = df['codmunres'].astype('Int64') 


df.head()


,origem,tipobito,dtobito,natural,codmunnatu,dtnasc,idade,sexo,racacor,estciv,...,fontes,tpresginfo,tpnivelinv,nudiasinf,dtcadinf,morteparto,dtconcaso,fontesinf,altcausa,contador
0,1,2,2020-05-18 14:52:00,812,120010,1932-06-07,487,1,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603
1,1,2,2020-05-20 21:15:00,812,120010,1952-03-16,468,2,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,604
2,1,2,2020-05-21 12:00:00,812,120010,1961-02-17,459,2,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,605
3,1,2,2020-05-21 12:33:00,812,120010,1942-08-10,477,2,4,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,606
4,1,2,2020-05-22 07:30:00,812,120030,1936-04-13,484,2,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607


In [15]:
df['dtconcaso']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
             ...    
1556819          NaN
1556820    9022021.0
1556821          NaN
1556822          NaN
1556823          NaN
Name: dtconcaso, Length: 1556824, dtype: float64

## Conclusões parciais : 

Até aqui, tudo está ok ? 

In [26]:
#df['ocup']
df = df.where(pd.notnull(df), None)

In [28]:
df3 = df.astype(object).replace(np.nan, 'None')
df3.head()

: 

: 

0          14:52:00
1          21:15:00
2          12:00:00
3          12:33:00
4          07:30:00
             ...   
1556819    20:20:00
1556820    12:20:00
1556821         NaT
1556822    21:00:00
1556823    16:32:00
Name: teste, Length: 1556824, dtype: object